# **Import libraries**

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import keras.utils as image
from sklearn.preprocessing import OneHotEncoder
from keras.src.utils import to_categorical,normalize
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
import keras.layers as layers
from keras.models import Model

## Load Data ##

In [2]:
train_path = "/kaggle/input/dataset/dataset/train/"
test_path = "/kaggle/input/dataset/dataset/test/"

In [3]:
IMG_SIZE = 250
classes = ['1', '2', '3', '4', '5']
Input_Shape = (IMG_SIZE, IMG_SIZE, 3)
images = []
labels = []
for c in classes:
    for img in os.listdir(train_path + c):
        img = cv2.imread(train_path + c + '/' + img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        images.append(img)
        labels.append(int(c))
        
images = np.array(images, dtype='float32')/255.0        

# Encoding labels 

In [4]:
onehotencoder = OneHotEncoder()
label = np.array(labels).reshape(-1,1)
labels = onehotencoder.fit_transform(label).toarray()

# Split Images and labels to train & validation 

In [5]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, shuffle=True, random_state=42)

## Augmentation For Train ##

In [6]:
training_datagen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    fill_mode="nearest",
)

training_datagen.fit(X_train)

# **DenseNet Model**

In [7]:
BATCH_SIZE = 64
NUM_EPOCHS = 100

In [ ]:
# Initializing the CNN
AlexNet = Sequential([
    Conv2D(96, 11, strides = (4, 4), padding ='valid', input_shape=(250, 250, 3), activation = 'relu'),
    MaxPooling2D(pool_size = (3, 3), strides = (2, 2), padding = 'valid'),
    BatchNormalization(),
    Conv2D(256, 5, strides = (1, 1), padding='valid', activation = 'relu'),
    Conv2D(256, 5, strides = (1, 1), padding='valid', activation = 'relu'),
    MaxPooling2D(pool_size = (3, 3), strides = (2, 2), padding='valid'),
    BatchNormalization(),
    
    Conv2D(384, 3, strides = (1, 1), padding='valid', activation = 'relu'),
    Conv2D(384, 3, strides = (1, 1), padding='valid', activation = 'relu'),
   
    Conv2D(256, 3, strides = (1, 1), padding='valid', activation = 'relu'),
    MaxPooling2D(pool_size = (3, 3), strides = (2, 2), padding = 'valid'),
    BatchNormalization(),
    
    # Flattening Step
    Flatten(),

    # Full Connection Step
    Dense(units = 4096, activation = 'relu'),
    Dropout(0.4),
    BatchNormalization(),
    Dense(units = 4096, activation = 'relu'),
    Dropout(0.4),
    BatchNormalization(),
    Dense(units = 1000, activation = 'relu'),
    Dropout(0.2),
    BatchNormalization(),
    Dense(units = 5, activation = 'softmax')
])

# Compile Model

In [9]:
AlexNet.compile(optimizer='adam',
              loss=tf.losses.categorical_crossentropy,
              metrics=['accuracy'])

# Fit Model

In [ ]:
NUM_EPOCHS = 100
history=AlexNet.fit(training_datagen.flow(X_train, y_train, batch_size=60),
         validation_data=training_datagen.flow(X_val, y_val),epochs=NUM_EPOCHS,steps_per_epoch=130)

# Plot the training and validation accuracy and loss at each epoch # 

In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
score = AlexNet.evaluate(X_val, y_val, verbose = 0 )
print("validation accuracy: ", score[1])
AlexNet.summary()

# Confusion Matrix

In [ ]:
# If your test_labels are one-hot encoded, convert them to integer labels
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
import numpy as np

val_integer_labels = np.argmax(y_val, axis=1)

# Get model predictions
predictions = AlexNet.predict(X_val)

# If your predictions are one-hot encoded, convert them to integer labels
predicted_labels = np.argmax(predictions, axis=1)

# Compute the confusion matrix
cm = confusion_matrix(val_integer_labels, predicted_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
#plt.show()

# **Test**

In [ ]:
X_test = []
image_coulmn=[]

for img in os.listdir(test_path):
    image_coulmn.append(int((img.split('.'))[0]))
    img = cv2.imread(test_path + img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    # Preprocess the image if needed (e.g., resizing)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    X_test.append(img)

# Convert the list of images to a NumPy array and normalization it
X_test = np.array(X_test)/255.0


In [ ]:
Output=AlexNet.predict(X_test, batch_size=16, verbose=0)

In [ ]:
Predition_classes=[]
for predict in Output:
    index=np.argmax(predict)
    Predition_classes.append(index+1)

In [ ]:
Predition_classes

In [ ]:
# # Create a dictionary with two columns
# data = {'image_id': image_coulmn,
#         'label': Predition_classes}

# # Create a DataFrame from the dictionary
# df = pd.DataFrame(data)

# # Print the DataFrame
# print(df)

# df.to_csv('file1.csv',header=False,index=False)
